In [0]:
import tensorflow as tf

In [4]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteYR8S1M/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteYR8S1M/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteYR8S1M/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [0]:
import numpy as np
train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s,l in train_data:

  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
# print(training_labels)

for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())
  
# training_labels_final = np.array(training_labels)
# testing_labels_final = np.array(testing_labels)

In [19]:
vocab_size = 10000
embedding_dim = 100
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)
print(padded[0])

tokenizer.fit_on_texts(testing_sentences)
word_index = tokenizer.word_index
sequences_test = tokenizer.texts_to_sequences(testing_sentences)
padded_test = pad_sequences(sequences_test,maxlen=max_length, truncating=trunc_type)
print(padded_test[0])

[   0    0   59   12   14   35  439  400   18  174   29    1    9   33
 1378 3401   42  496    1  197   25   88  156   19   12  211  340   29
   70  248  213    9  486   62   70   88  116   99   24 5740   12 3317
  657  777   12   18    7   35  406 8228  178 2477  426    2   92 1253
  140   72  149   55    2    1 7525   72  229   70 2962   16    1 2880
    1    1 1506 4998    3   40 3947  119 1608   17 3401   14  163   19
    4 1253  927 7986    9    4   18   13   14 4200    5  102  148 1237
   11  240  692   13   44   25  101   39   12 7232    1   39 1378    1
   52  409   11   99 1214  874  145   10]
[  59   44   25  108   13   97 3881   18  760 5420   10   14  319    5
    2  598  357   18 1663 1416    1   18  625 9063 4804    1  737    6
   13 1026    1    1  431  503    1    4    1  349 3725   20  233    3
   15 5995    3   15 1669   15  102    5    2 3987  101   11 1460 1475
   12  251  235   11  212    2  373 5320    3   62   96   11  175  106
   11 1475  183   12  251   37    6

In [21]:
temp = []
for s in padded:
  l=[]
  for i in s:
    l.append(i)
  temp.append(l)
print(temp[0])
train_context_words=[]
train_target_words =[]

for w in temp:
  for i in range(3,len(w)-3):
    train_target_words.append(w[i])
    temp2 = []
    for j in range(i-3,i+4):
      temp2.append(w[j])
    temp2.remove(w[i])
    train_context_words.append(temp2)
print(train_target_words[0])
print(train_context_words[0])

temp = []
for s in padded_test:
  l=[]
  for i in s:
    l.append(i)
  temp.append(l)
print(temp[0])
test_context_words=[]
test_target_words =[]

for w in temp:
  for i in range(3,int(len(w)/4)-3):
    test_target_words.append(w[i])
    temp2 = []
    for j in range(i-3,i+4):
      temp2.append(w[j])
    temp2.remove(w[i])
    test_context_words.append(temp2)
print(test_target_words[5])
print(test_context_words[5])

train_target_words = np.array(train_target_words)
test_target_words = np.array(test_target_words)
train_context_words = np.array(train_context_words)
test_context_words = np.array(test_context_words)

print(np.shape(train_target_words))

[0, 0, 59, 12, 14, 35, 439, 400, 18, 174, 29, 1, 9, 33, 1378, 3401, 42, 496, 1, 197, 25, 88, 156, 19, 12, 211, 340, 29, 70, 248, 213, 9, 486, 62, 70, 88, 116, 99, 24, 5740, 12, 3317, 657, 777, 12, 18, 7, 35, 406, 8228, 178, 2477, 426, 2, 92, 1253, 140, 72, 149, 55, 2, 1, 7525, 72, 229, 70, 2962, 16, 1, 2880, 1, 1, 1506, 4998, 3, 40, 3947, 119, 1608, 17, 3401, 14, 163, 19, 4, 1253, 927, 7986, 9, 4, 18, 13, 14, 4200, 5, 102, 148, 1237, 11, 240, 692, 13, 44, 25, 101, 39, 12, 7232, 1, 39, 1378, 1, 52, 409, 11, 99, 1214, 874, 145, 10]
12
[0, 0, 59, 14, 35, 439]
[59, 44, 25, 108, 13, 97, 3881, 18, 760, 5420, 10, 14, 319, 5, 2, 598, 357, 18, 1663, 1416, 1, 18, 625, 9063, 4804, 1, 737, 6, 13, 1026, 1, 1, 431, 503, 1, 4, 1, 349, 3725, 20, 233, 3, 15, 5995, 3, 15, 1669, 15, 102, 5, 2, 3987, 101, 11, 1460, 1475, 12, 251, 235, 11, 212, 2, 373, 5320, 3, 62, 96, 11, 175, 106, 11, 1475, 183, 12, 251, 37, 6, 1100, 1, 644, 7, 4387, 1, 4, 1, 349, 7, 37, 9284, 373, 5, 1455, 1, 13, 30, 64, 28, 6, 855, 178

In [22]:
print(np.shape(train_context_words))
print(np.shape(test_context_words))
print(np.shape(test_target_words))

(2850000, 6)
(600000, 6)
(600000,)


In [23]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(train_context_words[2]))
print(training_sentences[0])

b this was friend keep for
b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."


In [26]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=6),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(vocab_size, activation='softmax')
])
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 6, 100)            1000000   
_________________________________________________________________
flatten_3 (Flatten)          (None, 600)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 3606      
_________________________________________________________________
dense_7 (Dense)              (None, 10000)             70000     
Total params: 1,073,606
Trainable params: 1,073,606
Non-trainable params: 0
_________________________________________________________________


In [27]:
num_epochs = 5
model.fit(train_context_words, train_target_words,batch_size = 10000, epochs=num_epochs, validation_data=(test_context_words, test_target_words))

Epoch 1/5
285/285 [==============================] - 24s 85ms/step - loss: 6.8669 - accuracy: 0.0378 - val_loss: 5.7011 - val_accuracy: 0.1303
Epoch 2/5
285/285 [==============================] - 24s 85ms/step - loss: 6.0843 - accuracy: 0.0791 - val_loss: 5.5939 - val_accuracy: 0.1747
Epoch 3/5
285/285 [==============================] - 24s 85ms/step - loss: 5.8865 - accuracy: 0.1280 - val_loss: 5.5335 - val_accuracy: 0.2048
Epoch 4/5
285/285 [==============================] - 24s 84ms/step - loss: 5.4700 - accuracy: 0.1794 - val_loss: 5.6322 - val_accuracy: 0.2139
Epoch 5/5
285/285 [==============================] - 24s 85ms/step - loss: 5.1772 - accuracy: 0.2109 - val_loss: 5.7829 - val_accuracy: 0.2205


In [28]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

(10000, 100)


In [0]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')